# Curso de Optimización (DEMAT)
## Tarea 9

| Descripción:                         | Fechas               |
|--------------------------------------|----------------------|
| Fecha de publicación del documento:  | **Abril 28, 2022**   |
| Fecha límite de entrega de la tarea: | **Mayo   8, 2022**   |


### Indicaciones

- Envie el notebook que contenga los códigos y las pruebas realizadas de cada ejercicio.
- Si se requiren algunos scripts adicionales para poder reproducir las pruebas,
  agreguelos en un ZIP junto con el notebook.
- Genere un PDF del notebook y envielo por separado.

---

## Ejercicio 0 (0 puntos)

- Empezar a buscar un tema para el proyecto final del curso.
- En esta tarea no hay que poner la descripción del proyecto.
  Sólo buscar el tema y tenerlo listo para su entrega en la
  siguiente tarea.
- La fecha límite para mandar la descripción del proyecto
  se tiene que mandar es el domingo 17 de mayo. 



```



```
---


## Ejercicio 1. (2 puntos)

Programar las siguientes funciones y sus gradientes, de modo que dependan de la dimensión $n$ de la variable $\mathbf{x}$:


- Función "Tridiagonal 1" generalizada

$$  f(x) = \sum_{i=1}^{n-1} (x_i + x_{i+1} - 3)^2 + (x_i - x_{i+1} + 1)^4  $$


- Función generalizada de Rosenbrock

$$  f(x) = \sum_{i=1}^{n-1} 100(x_{i+1} - x_i^2)^2 + (1 - x_{i} )^2  $$


In [248]:
# Implementación de la funciones y sus gradientes
import numpy as np
from numpy import linalg as LA
from numpy.linalg import eigvals
import sys

def Tridiagonal1(x):
    return np.sum( (x[:-1] + x[1:] - 3)**2 + (x[:-1] - x[1:] + 1)**4)

def gradTridiagonal1(x):
    n = x.size
    g = np.zeros(n)
    g[1:-1] = 2*(x[:-2] + x[1:-1] -3) - 4*(x[:-2] - x[1:-1] + 1)**3 + 2*(x[1:-1] + x[2:] -3) + 4*(x[1:-1] - x[2:] + 1)**3
    g[0] = 2*(x[0] + x[1] - 3) + 4*(x[0] - x[1] + 1)**3
    g[n-1] = 2*(x[n-2] + x[n-1] - 3) - 4*(x[n-2] - x[n-1] + 1)**3
    return g

def Rosenbrock(y):
    x = y.T
    return np.sum( 100*(x[1:] - x[:-1]**2)**2 + (1-x[:-1])**2)

def gradRosenbrock(x):
    n = x.size
    g = np.zeros(n)
    g[1:-1] = 200*(x[1:-1] -x[:-2]**2) -400*x[1:-1]*(x[2:]-x[1:-1]**2)-2*(1-x[1:-1])
    g[0] = 200*(x[1]-x[0]**2)*(-2*x[0]) - 2*(1-x[0])
    g[n-1] = 200*(x[n-1]-x[n-2]**2)
    return g



```


```
---


## Ejercicio 1 (8 puntos)

Programar y probar el método BFGS modificado.


1. Programar el algoritmo descrito en la diapositiva 16 de la clase 23.
   Agregue una variable $res$ que indique si el algoritmo terminó
   porque se cumplió que la magnitud del gradiente es menor que la toleracia
   dada.
2. Probar el algoritmo con las funciones del Ejercicio 1
   con la matriz $H_0$ como la matriz identidad y el 
   punto inicial $x_0$ como:

- La función generalizada de Rosenbrock: 

$$ x_0 = (-1.2, 1, -1.2, 1, ..., -1.2, 1) \in \mathbb{R}^n$$

- La función Tridiagonal 1 generalizada: 

$$ x_0 = (2,2, ..., 2) \in \mathbb{R}^n $$
  
  Pruebe el algoritmo con la dimensión $n=2, 10 , 100$.

3. Fije el número de iteraciones máximas a $N=50000$, 
   y la tolerancia $\tau = \epsilon_m^{1/3}$, donde $\epsilon_m$
   es el épsilon máquina, para terminar las iteraciones 
   si la magnitud del gradiente es menor que $\tau$.
   En cada caso, imprima los siguiente datos:
   
- $n$,
- $f(x_0)$, 
- Usando la variable $res$, imprima un mensaje que indique si
  el algoritmo convergió,
- el  número $k$ de iteraciones realizadas,
- $f(x_k)$,
- la norma del vector $\nabla f_k$, y
- las primeras y últimas 4 entradas del punto $x_k$ que devuelve el algoritmo.
  

### Solución:

In [249]:
# En esta celda puede poner el código de las funciones
# o poner la instrucción para importarlas de un archivo .py
def Backtraking(f, fk, gk, xk, pk, a0, rho, c):
    a = a0
    while f(xk + a*pk) > fk +c*a*gk.T@pk:
        a = rho*a
    return a

def BFGS(f, g, H, x0, N, tol):
    res = 0
    h = 0.00001
    xk = x0.copy()
    n = x0.size
    Hk = H.copy()      #Aquí debería estar Hk(xk)
    I = np.identity(n)
    for k in range(N):
      gk = g(f, xk, h) #####Por ser el gradiente por dif Fin
      fk = f(xk)
      norm = LA.norm(gk)
      if norm < tol:
        res = 1
        break
      pk = -Hk@gk
      if pk.T@gk > 0:
        lamb1 = 0.00001 + (pk.T@gk)/(gk.T@gk)
        Hk = Hk + lamb1*I
        pk = pk - lamb1*gk    ##Condiciones de Wolfe
      ak = Backtraking(f, fk, gk, xk, pk, 2, 0.5, 0.0001)
      xk1 = xk.copy()        #xk
      xk = xk + ak*pk #xk+1
      sk = ak*pk
      gk1 = gk.copy()
      gk = g(f, xk, h)   #####Por ser el gradiente por dif Fin
      yk = gk - gk1   
      if yk.T@sk <= 0:
        lamb2 = 0.00001 - (yk.T@sk)/(sk.T@sk)
        Hk = Hk + lamb2*I
      else:
        rho_k = 1/(yk.T@sk)
        Hk = (I - rho_k*sk@yk.T)@Hk@(I - rho_k*yk@sk.T) + rho_k*sk@sk.T
    print(sk, yk, ak, pk, gk)
    return xk, fk, gk, k, res


In [250]:
# Pruebas realizadas 

def vectorx0Ros(n):
  x = np.zeros(n)
  for k in range( int(n/2)):
    x[2*k] = -1.2
    x[2*k+1] = 1
  return x

def vectorx0tri(n):
  x = np.zeros(n)
  for k in range(n):
    x[k] = 2
  return x


In [251]:
x0 = vectorx0Ros(4)
print(x0)
#print(Tridiagonal1(x0))
#print(gradTridiagonal1(x0))

#print(Rosenbrock(x0))
#print(gradRosenbrock(x0))
def fR(x):
  f=0
  for i in range(len(x)-1):
    f += 100 * (x[i+1] - x[i]**2)**2 + (1-x[i])**2
    return f

def GradDifFin(f, x, h):
    n = np.size(x)
    #print(n)
    Mx = np.tile(x , (n,1) ) #Repite el vector x, n veces.
    #print(Mx)
    eval = Mx + h*np.identity(n) 
    #print(eval)
    partial = (1/h)*(f(eval) - f(x))
    #print(f(eval))
    return partial

def function(x):
    xT = x
    return 100 * (xT[1] - xT[0]**2)**2 + (1-xT[0])**2 + 100 * (xT[2] - xT[1]**2)**2 + (1-xT[1])**2 + 100 * (xT[3] - xT[2]**2)**2 + (1-xT[2])**2 
    

#print(GradDifFin(Rosenbrock, x0, 0.00001))
#x1 = np.array([1, 2, 3, 4])


print(Rosenbrock(x0))
print(fR(x0))
print(function(x0))

print(GradDifFin(Rosenbrock, x0, 0.00001))
print(GradDifFin(function, x0, 0.00001))


[-1.2  1.  -1.2  1. ]
532.4000000000001
24.199999999999996
532.4000000000001
159719832.82470995
[ 1.57298728e+08 -5.32400000e+07  1.57298200e+08 -5.32400000e+07]


In [252]:
#Función de prueba

def ProbarRos(f, g, n, N, tol):
    x0 = vectorx0Ros(n)
    H = np.identity(n)
    xk, fk, gk, k, res = BFGS(f, g, H, x0, N, tol)
    if res == 1:
        print('¡Convergió! :)')
    else:
        print('¡No convergió! :(')
    print('Valor n:', n)
    print('f(x0):', f(x0))
    print('Valor k:', k)
    print('xk:', xk[:2], '...', xk[n-2:])
    print('f(xk):', fk)
    print('||fk||:', LA.norm(fk))
    print('res:', res)

def ProbarTri(f, g, n, N, tol):
    x0 = vectorx0tri(n)
    H = np.identity(n)
    xk, fk, gk, k, res = BFGS(f, g, H, x0, N, tol)
    if res == 1:
        print('¡Convergió! :)')
    else:
        print('¡No convergió! :(')
    print('Valor n:', n)
    print('f(x0):', f(x0))
    print('Valor k:', k)
    print('xk:', xk[:2], '...', xk[n-2:])
    print('f(xk):', fk)
    print('||fk||:', LA.norm(fk))
    print('res:', res)


```


```
---

In [253]:

#Pruebas
eps = sys.float_info.epsilon
tol = eps**(1/3)

import warnings
warnings.simplefilter('error', RuntimeWarning)

x0 = vectorx0Ros(4)
grad = GradDifFin(fR, x0, 0.00001)
print('Pruebas para la función de Rosenbrock')
#ProbarRos(Rosenbrock, gradRosenbrock, 4, 50000, tol)
print(' ')
#ProbarRos(Rosenbrock, GradDifFin(fR, x0, 0.000001), 4, 50000, tol)
print(' ')
ProbarRos(fR, GradDifFin, 2, 10000, tol)

Pruebas para la función de Rosenbrock
 
 
[-2.40832096e-19  8.60130770e-21] [0. 0.] 3.2311742677852644e-27 [-74533923.6649152    2661975.79890159] [67758728.41104993 -2419999.999     ]
¡No convergió! :(
Valor n: 2
f(x0): 24.199999999999996
Valor k: 9999
xk: [-1.2  1. ] ... [-1.2  1. ]
f(xk): 24.199999999999996
||fk||: 24.199999999999996
res: 0


In [254]:
print('Pruebas para la función de Tridiagonal')
ProbarTri(Tridiagonal1, gradTridiagonal1, 2, 50000, tol)
print(' ')
ProbarTri(Tridiagonal1, gradTridiagonal1, 10, 50000, tol)
print(' ')
ProbarTri(Tridiagonal1, gradTridiagonal1, 100, 50000, tol)


Pruebas para la función de Tridiagonal


TypeError: ignored